In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [7]:
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'us_census\\acsbr-015.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.1 Public policy changes included \nthe renewal of the Public Health Emergency, wh

In [8]:
HF_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5", #senetence-transforers/all-MiniLM-16-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},

)

C:\Users\9luis\AppData\Local\Temp\ipykernel_9420\1588872652.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  HF_embeddings=HuggingFaceBgeEmbeddings(
c:\Users\9luis\OneDrive\Desktop\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\9luis\OneDrive\Desktop\Langchain\venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not 

In [10]:
import numpy as np
np.array(HF_embeddings.embed_query(final_documents[0].page_content))

(384,)

In [11]:
vectorstore=FAISS.from_documents(final_documents[:120], HF_embeddings)


In [24]:
query="What is Health Insurance Coverage?"
relevant_documents=vectorstore.similarity_search(query)

print(relevant_documents[0].page_content)
# print("------------------------------------------------")
# print(relevant_documents[1].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.
-----------------------------------

In [27]:
retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={'k':3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023D3296C980> search_kwargs={'k': 3}


In [29]:
import os
from dotenv import load_dotenv
load_dotenv()

hugginface_token=os.environ['HUGGINGFACEHUB_API_TOKEN']


In [37]:
from langchain_community.llms import HuggingFaceHub

hf = HuggingFaceHub(
    # repo_id="google/flan-t5-base",  # Using a smaller model that works with the API
    # repo_id="microsoft/DialoGPT-medium",
    repo_id="tiiuae/falcon-7b-instruct", #resulted in best output
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

c:\Users\9luis\OneDrive\Desktop\Langchain\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


'What is the health insurance coverage?\nHealth insurance coverage is a type of insurance that pays for medical expenses that are not covered by your primary health insurance. This can include things like doctor visits, prescriptions, and hospital stays. Some health insurance plans may also cover preventive care, such as annual check-ups and screenings.'

In [47]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# Create the pipeline with the correct model type
model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
# Create the pipeline
pipe = pipeline(
    "text2text-generation",  # Use text2text-generation instead of text-generation
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=0,
    device="cpu"
)

# Create the LangChain pipeline
hf = HuggingFacePipeline(pipeline=pipe)
llm = hf
# hf = HuggingFacePipeline.from_model_id(
#     # model_id="tiiuae/falcon-7b-instruct",
#     model=model,
#     tokenizer=tokenizer,
#     # model_id = "CohereForAI/c4ai-command-r-v01",
#     # model_id="mistralai/Mistral-7B-v0.1",
#     task="text-generation",
#     pipeline_kwargs={"temperature": 0, "max_new_tokens": 300, "device": "cpu"}

# )

# llm = hf
# llm.invoke(query)

Device set to use cpu
C:\Users\9luis\AppData\Local\Temp\ipykernel_9420\4223237203.py:19: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf = HuggingFacePipeline(pipeline=pipe)


In [49]:
prompt_templates="""
Use the following pieve of context to answer the questions asked.
Please try to provide the answer only based on the context

{context}
Questions:{question}

Helpful Answers:
"""

In [51]:
prompt=PromptTemplate(template=prompt_templates,input_variables=["context","question"])

In [53]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [54]:
query="""
DIFFERENCES IN THE UNINSURED RATE BY STATE IN 2022
"""

In [55]:
result = retrievalQA.invoke({"query": query})
print(result['result'])

Token indices sequence length is longer than the specified maximum sequence length for this model (810 > 512). Running this sequence through the model will result in indexing errors
c:\Users\9luis\OneDrive\Desktop\Langchain\venv\Lib\site-packages\transformers\generation\configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


ACS
